In [8]:
import os

import pandas as pd
import psycopg
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

TABLE_NAME = 'users_churn'

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = 'test_exp1'
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = "churn_model_nikolaibogomolov"
######





In [9]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net',
    "port": 6432,
    "dbname": 'playground_mle_20240525_947977af38',
    "user": 'mle_20240525_947977af38',
    "password": 'c12f5c4ec96c4d65a4d2f665c6fd9d73',
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df = df.dropna()

df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
4,1001,8357-EQXFO,2019-04-01,2019-11-01,Month-to-month,Yes,Electronic check,95.35,660.90,Fiber optic,...,Yes,No,Yes,Yes,Female,0,No,No,No,1
5,1002,1989-PRJHP,2017-08-01,2019-11-01,Month-to-month,Yes,Electronic check,75.50,1893.95,Fiber optic,...,No,No,No,No,Male,1,Yes,No,Yes,1


In [10]:
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]
obj_df = df.select_dtypes(include="object")

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(
    categories='auto',
    handle_unknown='ignore',  # Игнорировать неизвестные категории
    sparse_output=False,  # Вывод в виде обычного массива
    drop='first',  # Удаление первой категории
    max_categories=10  # Максимальное количество категорий (опционально)
)

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
#encoded_features = encoder_oh.fit_transform(df[cat_columns])
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy())

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
 # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out(cat_columns))

# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)



,customer_id,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,...,partner,dependents,multiple_lines,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,internet_service_Fiber optic,gender_Male
4,8357-EQXFO,Month-to-month,Yes,Electronic check,Fiber optic,No,No,Yes,No,Yes,...,No,No,No,1.0,0.0,0.0,0.0,1.0,1.0,0.0
5,1989-PRJHP,Month-to-month,Yes,Electronic check,Fiber optic,No,No,No,No,No,...,Yes,No,Yes,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [11]:

num_columns = ["monthly_charges", "total_charges"]
num_df = df[num_columns]

In [12]:


n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None


# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([num_df, encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(    encoded_features, 
    columns=encoder_q.get_feature_names_out(num_columns)
)
    
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))

encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# # PolynomialFeatures
# encoder_pol = PolynomialFeatures(degree=degree)
# encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())

# encoded_df = pd.DataFrame( encoded_features, 
#     columns=encoder_pol.get_feature_names_out(num_columns))
                          
# encoded_df.columns = [encoded_df.columns[1 + len(num_columns):]]
# num_df = pd.concat([num_df, encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame( encoded_features, 
    columns=encoder_kbd.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


num_df.head(2)



,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,total_charges_sp_2,total_charges_sp_3,total_charges_sp_4,total_charges_sp_5,monthly_charges_q_100,total_charges_q_100,monthly_charges_robust,total_charges_robust,monthly_charges_bin,total_charges_bin
4,95.35,660.90,0.000000,0.002112,0.186388,0.598552,0.209770,0.003178,0.000000,0.023461,0.388126,0.519622,6.877732e-02,0.000013,0.854912,0.90131,0.702528,1.667648,3.0,2.0
5,75.50,1893.95,0.025832,0.399520,0.510814,0.063827,0.000007,0.000000,0.035137,0.437076,0.478720,0.049066,1.261749e-07,0.000000,0.044148,0.26911,-1.591178,-0.274583,0.0,0.0


In [6]:
numeric_transformer = ColumnTransformer(
    transformers=[('spl', encoder_spl, num_columns),
    ('q', encoder_q, num_columns), ('rb', encoder_rb, num_columns), 
     ('kbd', encoder_kbd, num_columns)]
)

categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)])

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, num_columns),              
    ('cat', categorical_transformer, cat_columns)], n_jobs=-1)

encoded_features = preprocessor.fit_transform(df)

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

df = pd.concat([df, transformed_df], axis=1)
transformed_df.head(2)

,num__spl__monthly_charges_sp_0,num__spl__monthly_charges_sp_1,num__spl__monthly_charges_sp_2,num__spl__monthly_charges_sp_3,num__spl__monthly_charges_sp_4,num__spl__monthly_charges_sp_5,num__spl__total_charges_sp_0,num__spl__total_charges_sp_1,num__spl__total_charges_sp_2,num__spl__total_charges_sp_3,...,num__rb__total_charges,num__kbd__monthly_charges,num__kbd__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__gender_Male
0,0.000000,0.005757,0.250948,0.591477,0.150852,0.000967,0.022473,0.383100,0.523373,0.071036,...,-0.094402,3.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.000014,0.069284,0.520477,0.386990,0.023235,0.000000,0.004448,0.231955,0.595760,0.166434,...,0.413130,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [13]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJEryDs7iScbshPQ7BaUhes'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCMl4tpgidAdLCoZRZ1lURSmOpgRQ12KhwO_tJkr'

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, "column_transformer") 

2024/08/21 12:50:21 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [14]:
transformed_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586 entries, 0 to 1585
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   num__spl__monthly_charges_sp_0               1586 non-null   float64
 1   num__spl__monthly_charges_sp_1               1586 non-null   float64
 2   num__spl__monthly_charges_sp_2               1586 non-null   float64
 3   num__spl__monthly_charges_sp_3               1586 non-null   float64
 4   num__spl__monthly_charges_sp_4               1586 non-null   float64
 5   num__spl__monthly_charges_sp_5               1586 non-null   float64
 6   num__spl__total_charges_sp_0                 1586 non-null   float64
 7   num__spl__total_charges_sp_1                 1586 non-null   float64
 8   num__spl__total_charges_sp_2                 1586 non-null   float64
 9   num__spl__total_charges_sp_3                 1586 non-null   float64
 10  

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1586 entries, 4 to 7041
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1586 non-null   int64         
 1   customer_id        1586 non-null   object        
 2   begin_date         1586 non-null   datetime64[ns]
 3   end_date           1586 non-null   datetime64[ns]
 4   type               1586 non-null   object        
 5   paperless_billing  1586 non-null   object        
 6   payment_method     1586 non-null   object        
 7   monthly_charges    1586 non-null   float64       
 8   total_charges      1586 non-null   float64       
 9   internet_service   1586 non-null   object        
 10  online_security    1586 non-null   object        
 11  online_backup      1586 non-null   object        
 12  device_protection  1586 non-null   object        
 13  tech_support       1586 non-null   object        
 14  streaming_tv 

In [16]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Разделяем данные на признаки и целевую переменную
X = transformed_df
y = df['target'][:1586]

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем модель линейной регрессии
model = LinearRegression()

# Обучаем модель на обучающей выборке
model.fit(X_train, y_train)

# Делаем прогнозы на тестовой выборке
y_pred = model.predict(X_test)

# Вычисляем среднеквадратичную ошибку
mse = mean_squared_error(y_test, y_pred)

# Выводим результаты
print(f'Среднеквадратичная ошибка: {mse}')

# Выводим уравнение регрессии
print(f'Уравнение регрессии: y = {model.intercept_} + {model.coef_[0]}*x1 + {model.coef_[1]}*x2')


Среднеквадратичная ошибка: 0.0
Уравнение регрессии: y = 1.0 + 0.0*x1 + 0.0*x2


In [17]:
#experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
prediction = y_pred


pip_requirements = 'requirements.txt'
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}



experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id



with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id, nested=True) as run:
    run_id = run.info.run_id
    print(run_id)

    model_info = mlflow.sklearn.log_model( 
        sk_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature = signature,
        pip_requirements=pip_requirements,
        input_example = input_example,
        metadata = metadata,
        await_registration_for=60       
    )

b49084d1d5ef4ef4b72de3f4980f46be


Successfully registered model 'churn_model_nikolaibogomolov'.
2024/08/21 12:58:45 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: churn_model_nikolaibogomolov, version 1
Created version '1' of model 'churn_model_nikolaibogomolov'.
